# Variabelutvinning

Her ser vi på data fra filmrangeringer. Målet er å predikere hva slage rangering forskiellige brukere gir til filemer basert på data om filmene og brukere. 

Her har jeg lagd en enkel modell for det. Dere har tre oppgaver:

1. Les gjennom koden og prøv å forstå den. 
2. I grupper, lag en liste av minst 10 forskjellige mulige variabler (basert på de eksisterende data) som kunne hjelpe å forbedre prediksjonene. 
3. Implementer de tre variablene som du tror mest på at hjelper. 
4. Basert på resultatene, velg ut hvilke variabler du faktisk vil ha med.

In [1]:
# importer nødvendige pakker
import numpy as np
import pandas as pd
from datetime import datetime
#import pgeocode 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

In [2]:
# lese inn data 
df_ranking = pd.read_csv('./data/ratings.dat', sep='::', engine='python', header=None, 
                         names = ['UserID', 'FilmID', 'Ranking', 'Timestamp'])
df_ranking['Timestamp'] = pd.to_datetime(df_ranking['Timestamp'], unit='s')
df_user = pd.read_csv('./data/users.dat', sep='::', header=None, 
                      names=['UserID', 'Sex', 'Age', 'Job', 'Post code'], 
                      engine='python')
df_film = pd.read_csv('./data/movies.dat', sep='::', header=None,
                      names=['FilmID', 'Title', 'Genre'], 
                      encoding='latin-1', engine='python')

In [3]:
# se på ranking-data
df_ranking.head()

,UserID,FilmID,Ranking,Timestamp
0,1,1193,5,2000-12-31 22:12:40
1,1,661,3,2000-12-31 22:35:09
2,1,914,3,2000-12-31 22:32:48
3,1,3408,4,2000-12-31 22:04:35
4,1,2355,5,2001-01-06 23:38:11


In [4]:
# se på bruker-data
df_user.head()

,UserID,Sex,Age,Job,Post code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
# se på film-data
df_film.head()

,FilmID,Title,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [13]:
# kombiner data
df_combined = df_ranking.merge(df_user, how='left', on='UserID').merge(df_film, how='left', on='FilmID')
df_combined.sort_values(by='UserID', ascending=False).head(50)

,UserID,FilmID,Ranking,Timestamp,Sex,Age,Job,Post code,Title,Genre
1000208,6040,1097,4,2000-04-26 02:19:29,M,25,6,11106,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
999984,6040,2575,4,2001-08-10 14:29:22,M,25,6,11106,"Dreamlife of Angels, The (La Vie rêvée des ang...",Drama
999976,6040,1587,1,2000-04-26 02:32:54,M,25,6,11106,Conan the Barbarian (1982),Action|Adventure
999977,6040,1594,3,2000-07-28 23:56:39,M,25,6,11106,In the Company of Men (1997),Drama
999978,6040,3504,4,2000-06-14 08:37:37,M,25,6,11106,Network (1976),Comedy|Drama
999979,6040,1900,5,2000-07-28 23:52:32,M,25,6,11106,"Children of Heaven, The (Bacheha-Ye Aseman) (1...",Drama
999980,6040,3505,4,2000-04-25 23:07:36,M,25,6,11106,No Way Out (1987),Thriller
999981,6040,1912,3,2000-07-28 23:55:42,M,25,6,11106,Out of Sight (1998),Action|Crime|Romance
999982,6040,2571,4,2001-08-10 14:35:26,M,25,6,11106,"Matrix, The (1999)",Action|Sci-Fi|Thriller
999983,6040,300,2,2000-04-25 23:18:36,M,25,6,11106,Quiz Show (1994),Drama


In [7]:
# del data i trenings-, validerings-, og testdata
df_train, df_valtest = train_test_split(df_combined, test_size=0.3, random_state=0)
df_val, df_test = train_test_split(df_valtest, test_size=0.5, random_state=0)

In [8]:
# forberede y 
def prepare_y(df):
    return df.Ranking.values
def prepare_ys(dfs):
    return (prepare_y(df) for df in dfs)

Det er `prepare_X`-funksjonen som bør oppdateres, slik at X inneholder flere relevante kolonner som kan brukes til å predikere rangeringer. 

In [9]:
# forberede X 
def prepare_X(df):
    numeric_cols = ["Ranking","FilmID","Job"]
    return df.loc[:, numeric_cols]

def prepare_Xs(dfs):
    return (prepare_X(df) for df in dfs)

In [10]:
# faktiske dataforberedelse
X_train, X_val, X_test = prepare_Xs((df_train, df_val, df_test))
y_train, y_val, y_test = prepare_ys((df_train, df_val, df_test))

In [11]:
# se på X fra tranings-data 
X_train.head()

,Ranking,FilmID,Job
103224,2,1220,4
495518,5,1409,16
762996,4,344,19
611883,4,2336,12
531137,1,1726,7


In [12]:
# regresjon 
reg = LinearRegression()
scaler = StandardScaler()
imp = SimpleImputer(strategy='mean')
model = make_pipeline(imp, scaler, reg)

model.fit(X_train, y_train)
pred = model.predict(X_val).clip(min=1, max=5)
rmse = mean_squared_error(y_val, pred, squared=False)
print(rmse)

5.915086596300538e-16


Mulig variabler:

1. Ranking

For å gjette ranking, så burde man ha med ranking

2. FilmID 

En film er ikke bra eller dårlig, men gjennomsnittet av det alle stemmer viser hvor god filmen er.

3. Age 

Age kan ha noe å si, men da må man ha med ```FilmID``` siden noen filmer med noen id-er vil få en bedre score sammen med alder

4. Jobb

Jobb vil kunne ha noe å si.

5. Sex

Kan være at menn/kvinner gir forskjellig score (litt mindre/mer) score basert på kjønn.

6. UserID

Noen brukere kan generelt gi dårlig scores, eller bedre. Dette kan hjelp en del

7. Timestamp

Transform to just year




### Tror ikke blir bra:
PostCode - ikke verdi + ikke numeric
Timestamp - Skal ikke ha noe å si  + ikke numeric
Title - FilmID er mye bedre + ikke numeric
Genre - ikke numeric


**['Age']**

1.11

**['Age',"FilmID"]**

1.11

### RANKING ER VELDIG VIKTIG!!!

**['Age',"Ranking"]**

1.09776e-15

**['Age',"Ranking","FilmID"]**

9.845454659780244e-16

**["Ranking","FilmID","Job"]**

5.915086596300538e-16  ***minst***

**['Age',"Ranking","FilmID","Job"]**

1.1843520163624952e-15

**['Age',"Ranking","FilmID","Job","UserID"]**

1.3927981302580887e-15

**['Age',"Ranking","FilmID","Job","Postcode"]**

dont work



